In [10]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [12]:
data = pd.read_csv('s1_improvised.csv', header = None, engine = 'python', encoding = 'latin-1')
data.dropna(axis=0, how="all", thresh=None, subset=None, inplace=True)

data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,NO.,MA101,ma_internals,PH100,ph100_internals,BE10105,be10105_internals,BE110,be110_internals,BE103,be103_internals,EC100,ec100_internals,PH110,ph110_internals,EC110,ec110_internals,CS110,cs110_internals
1,1,0.0,25,0,23,0.0,35,8,96,5,36,0,30,9,86,9,85,7,75
2,2,6.0,41,7,46,7.0,47,7,92,7,40,0,37,10,97,9,87,10,98
3,3,10.0,47,10,50,6.0,47,8,74,9,46,8,50,10,97,9,86,10,97
4,5,6.0,31,9,41,5.0,42,7,78,9,42,7,43,10,94,10,97,9,87
5,6,10.0,45,7,46,0.0,40,6,82,7,41,6,39,10,93,10,95,10,98
6,7,9.0,46,9,47,8.0,48,8,79,8,45,9,47,10,95,10,94,10,97
7,8,5.0,25,6,40,0.0,37,0,79,7,37,0,40,10,96,9,88,10,94
8,9,6.0,31,6,34,5.0,35,0,68,6,38,5,38,10,95,10,91,10,93
9,10,0.0,24,4,24,0.0,38,7,80,7,41,0,32,8,83,8,83,10,94


In [20]:
x=pd.read_csv('s1_train.csv')
x

,NO.,MA101,PH100,BE10105,BE110,BE103,EC100,PH110,EC110,CS110
0,12,6.0,6,5.0,6,6,0,10,8,10
1,36,5.0,6,5.0,6,7,0,10,9,9
2,28,0.0,0,0.0,0,7,0,9,7,8
3,39,10.0,8,0.0,10,8,7,10,10,9
4,46,0.0,0,9.0,6,6,0,10,8,9
5,45,7.0,7,8.0,8,6,0,10,10,10
6,9,6.0,6,5.0,0,6,5,10,10,10
7,17,7.0,9,8.0,10,8,6,10,10,10
8,35,5.0,5,6.0,7,7,6,9,10,9
9,43,7.0,6,5.0,9,7,6,10,9,9


In [13]:
training_set = pd.read_csv('s1_train.csv')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('s1_test.csv')
test_set = np.array(test_set, dtype = 'int') 

In [14]:
print(training_set)

[[12  6  6  5  6  6  0 10  8 10]
 [36  5  6  5  6  7  0 10  9  9]
 [28  0  0  0  0  7  0  9  7  8]
 [39 10  8  0 10  8  7 10 10  9]
 [46  0  0  9  6  6  0 10  8  9]
 [45  7  7  8  8  6  0 10 10 10]
 [ 9  6  6  5  0  6  5 10 10 10]
 [17  7  9  8 10  8  6 10 10 10]
 [35  5  5  6  7  7  6  9 10  9]
 [43  7  6  5  9  7  6 10  9  9]
 [58  5  6  5  7  6  0 10 10  8]
 [22 10  8  7  9  6  7  9  8 10]
 [60  5  6  0  7  6  5 10 10  9]
 [42  8  7  6  7  7  7 10 10 10]
 [19  7  7  6 10  7  7  9  8 10]
 [ 7  9  9  8  8  8  9 10 10 10]
 [37  9  7  0  8  7  9 10 10  9]
 [20  7  6  9 10  8  6  9 10 10]
 [55  6  5  5  7  6  6 10 10 10]
 [25  0  6  0  6  7  5  9  7  9]
 [61  6  4  0  0  5  0  9  7  8]
 [10  0  4  0  7  7  0  8  8 10]
 [16 10 10  9 10  8  8 10  9 10]
 [33  8  6  0  0  9  5  9  9 10]
 [21  0  6  0  0  6  0 10  9 10]
 [49  5  6  0  7  7  0  9  9 10]
 [52  8  9  5  9  7  8 10 10 10]
 [ 2  6  7  7  7  7  0 10  9 10]
 [15  9  7  8 10  7  0 10  8 10]
 [50  0  0  0  0  7  0  9  8 10]
 [31  0  5

In [34]:
nb_students = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_courses = max(training_set.shape[1], test_set.shape[1])-1


In [39]:
# def convert(data):
#     new_data = []
#     for id_users in range(1, no_students + 1):
#         id_courses = data[:,1][data[:,0] == id_users]
#         id_ratings = data[:,2][data[:,0] == id_users]
#         ratings = np.zeros(no_movies)
#         print(ratings)
#         print(ratings.size)
#         ratings[id_courses - 1] = id_ratings
#         print(ratings)
#         print(ratings.size)
#         new_data.append(list(ratings))
#     return new_data

In [35]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [36]:
print(training_set)
print(test_set)

tensor([[12.,  6.,  6.,  5.,  6.,  6.,  0., 10.,  8., 10.],
        [36.,  5.,  6.,  5.,  6.,  7.,  0., 10.,  9.,  9.],
        [28.,  0.,  0.,  0.,  0.,  7.,  0.,  9.,  7.,  8.],
        [39., 10.,  8.,  0., 10.,  8.,  7., 10., 10.,  9.],
        [46.,  0.,  0.,  9.,  6.,  6.,  0., 10.,  8.,  9.],
        [45.,  7.,  7.,  8.,  8.,  6.,  0., 10., 10., 10.],
        [ 9.,  6.,  6.,  5.,  0.,  6.,  5., 10., 10., 10.],
        [17.,  7.,  9.,  8., 10.,  8.,  6., 10., 10., 10.],
        [35.,  5.,  5.,  6.,  7.,  7.,  6.,  9., 10.,  9.],
        [43.,  7.,  6.,  5.,  9.,  7.,  6., 10.,  9.,  9.],
        [58.,  5.,  6.,  5.,  7.,  6.,  0., 10., 10.,  8.],
        [22., 10.,  8.,  7.,  9.,  6.,  7.,  9.,  8., 10.],
        [60.,  5.,  6.,  0.,  7.,  6.,  5., 10., 10.,  9.],
        [42.,  8.,  7.,  6.,  7.,  7.,  7., 10., 10., 10.],
        [19.,  7.,  7.,  6., 10.,  7.,  7.,  9.,  8., 10.],
        [ 7.,  9.,  9.,  8.,  8.,  8.,  9., 10., 10., 10.],
        [37.,  9.,  7.,  0.,  8.,  7.,  

In [37]:

# Creating the architecture of the Neural Network
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_courses, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_courses)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)
`

In [39]:
# Training the SAE
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_students):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_courses/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data[0]*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))


RuntimeError: size mismatch, m1: [1 x 10], m2: [9 x 20] at /opt/conda/conda-bld/pytorch_1579022060824/work/aten/src/TH/generic/THTensorMath.cpp:136

In [70]:
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)

    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)

    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)

    def train(self, v0, vk, ph0, phk):
        self.W += torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)

In [73]:
nv = len(training_set[0])
nh = 19
batch_size = 31
rbm = RBM(nv, nh)

In [74]:
nb_users  = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1
        print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(32.9491)
epoch: 2 loss: tensor(32.6197)
epoch: 3 loss: tensor(32.6078)
epoch: 4 loss: tensor(32.6078)
epoch: 5 loss: tensor(32.6078)
epoch: 6 loss: tensor(32.6078)
epoch: 7 loss: tensor(32.6078)
epoch: 8 loss: tensor(32.6078)
epoch: 9 loss: tensor(32.6078)
epoch: 10 loss: tensor(32.6078)


In [75]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

IndexError: The shape of the mask [1, 19] at index 0 does not match the shape of the indexed tensor [0, 19] at index 0

In [77]:
vt

tensor([[62., 28.,  5., 33.,  0., 26.,  0., 94., 10., 94., 10., 23.,  5., 77.,
          7., 31.,  5., 63.,  6.]])

In [54]:
_,v

(tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1.]]), tensor([], size=(0, 19)))

In [56]:
vt

tensor([], size=(0, 19))